In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats

In [2]:
data = pd.read_csv("surveydata.csv",sep=',',header=0)
data.head()

,id,提交答卷时间,所用时间,来源,来源详情,来自IP,目前恋爱状态,恋爱次数,结婚意愿,4、您的理想结婚年龄为,...,性别,年龄,来自,户口,25、请问您目前从事的职业为,26、请问您的最高学历是（包括在读）,27、请问您是否为独生子女,28、请问您的月消费区间为,29、如果您对我们的调查结果感兴趣，可以留下您的电子邮箱地址，我们将在课题完成后向您发送研究报告,总分
0,1,2022/11/10 12:54,250秒,微信,NaN,202.120.235.202(上海-上海),单身，无恋爱打算,3,不知道，顺其自然,26-30,...,男,23,山东,城镇,全日制学生,硕士,是,5000-1万,(空),0
1,2,2022/11/10 12:54,183秒,微信,NaN,202.120.235.88(上海-上海),单身，无恋爱打算,0,不知道，顺其自然,26-30,...,男,21,上海,城镇,全日制学生,硕士,是,1000-3000,(空),0
2,3,2022/11/10 12:55,166秒,微信,NaN,223.104.210.217(上海-上海),单身，有恋爱打算,1,有结婚打算,26-30,...,男,23,上海,城镇,全日制学生,硕士,是,1000-3000,(空),0
3,4,2022/11/10 12:55,170秒,手机提交,直接访问,183.192.11.120(上海-上海),单身，有恋爱打算,3,有结婚打算,26-30,...,男,22,上海,城镇,全日制学生,本科,是,3000-5000,(空),0
4,5,2022/11/10 12:56,254秒,微信,NaN,101.82.242.152(上海-上海),单身，无恋爱打算,1,不知道，顺其自然,26-30,...,女,22,湖北,城镇,全日制学生,硕士,是,1000-3000,(空),0


In [3]:
data.columns

Index(['id', '提交答卷时间', '所用时间', '来源', '来源详情', '来自IP', '目前恋爱状态', '恋爱次数', '结婚意愿',
       '4、您的理想结婚年龄为', '5、请问您认为或您见到的促进结婚的因素有哪些', '6、请问您认为或您见到的抑制结婚的因素有哪些',
       '7、请问您对于以下婚姻行为的认同度为—婚前同居', '7、婚前性行为', '7、不婚', '7、丁克（不生孩子）', '7、同性恋行为',
       '7、闪婚行为', '7、非婚生育', '7、堕胎', '7、开放性关系', '意愿子女数', '理想子女数', '理想儿子', '理想女儿',
       '9、您认为生儿育女能给您带来的价值有（1为最高，6为最低）', '10、请问您见到的或认为的阻碍生育的因素有哪些',
       '11、请问您认为的可以提高生育意愿的政策有哪些', '12、请问您认为阻碍您生育的因素有哪些',
       '13、请问您认为的可以提高生育意愿的政策有哪些', '上网时长', '动漫', '综艺', '体育/电竞', '明星', '科技/数码',
       '网文', '游戏', '微博', '知乎', '豆瓣', '虎扑', '百度贴吧', '小红书', 'B站', '短视频',
       '朋友圈秀恩爱晒娃', '恋爱相亲育儿综艺', '国家生育政策', '专家教授发表看法', '娱乐圈情感八卦', '身边人经历',
       '艺术作品', '18、请问您上网时是否会分享自己的情感、恋爱及婚育观点？', '19、请问您是否会在朋友圈或社交媒体上秀恩爱？',
       '20、请问如果您有孩子（或者您已经有），您会在朋友圈或社交媒体上晒娃吗？', '性别', '年龄', '来自', '户口',
       '25、请问您目前从事的职业为', '26、请问您的最高学历是（包括在读）', '27、请问您是否为独生子女',
       '28、请问您的月消费区间为', '29、如果您对我们的调查结果感兴趣，可以留下您的电子邮箱地址，我们将在课题完成后向您发送研究报告',
       '总分'],
      dtype='object')

In [4]:
tmp_columns = list(data.columns)
tmp_columns[56] = 'gender'
circles = ['动漫','综艺','体育/电竞','明星','科技/数码','网文','游戏','微博','知乎','豆瓣','虎扑','百度贴吧','小红书','B站','抖音']
tmp_columns[31:46] = circles
tmp_columns[10] = 'marriage_pro'
tmp_columns[11] = 'marriage_con'
tmp_columns[26:30] = ['birth_con_1','birth_pro_1','birth_con_2','birth_pro_2']
data.columns = tmp_columns

In [5]:
reasons_bc = data['birth_con_1'].str.split(chr(9483)).explode().unique()[:9]
reasons_bp = data['birth_pro_1'].str.split(chr(9483)).explode().unique()[:11]
reasons_mc = data['marriage_con'].str.split(chr(9483)).explode().unique()[:11]
reasons_mp = data['marriage_pro'].str.split(chr(9483)).explode().unique()[:9]

In [6]:
for r in reasons_bc:
    data[r] = data['birth_con_1'].str.contains(r) | data['birth_con_2'].str.contains(r)
for r in reasons_bp:
    data[r] = data['birth_pro_1'].str.contains(r) | data['birth_pro_2'].str.contains(r)
for r in reasons_mc:
    data[r] = data['marriage_con'].str.contains(r)
for r in reasons_mp:
    data[r] = data['marriage_pro'].str.contains(r)

d:\anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.


In [9]:
reasons = list(reasons_bc)+list(reasons_bp)+list(reasons_mc)+list(reasons_mp)
reasons.remove('(跳过)')
reasons.remove('其他')

In [11]:
dt = data[['gender']+circles+reasons]
dt.head()

,gender,动漫,综艺,体育/电竞,明星,科技/数码,网文,游戏,微博,知乎,...,难以找到物质条件上满意的结婚对象,父母长辈对子女伴侣要求较高,婚姻能够提供情感、陪伴或精神寄托,婚姻可以带来物质经济保障,来自长辈的压力,生理需求,婚姻是人生的必需项,有生儿育女的期望,为了获得婚育政策上的优待,来自同龄人的压力
0,男,参与较少,不参与,参与较多,参与较少,参与较多,参与较少,参与较少,使用较多,使用较多,...,False,False,True,True,True,True,False,False,False,False
1,男,不参与,不参与,不参与,不参与,不参与,不参与,不参与,完全不使用,完全不使用,...,False,False,True,False,True,False,True,False,False,False
2,男,参与程度极高,不参与,参与较多,不参与,参与较少,参与较多,参与程度极高,使用较少,使用较多,...,False,False,True,False,False,True,False,True,False,False
3,男,参与较少,参与较多,参与较多,不参与,参与较少,参与较少,参与较多,完全不使用,使用较多,...,False,False,True,False,False,False,True,False,False,False
4,女,参与程度极高,参与较多,参与较少,参与较少,参与较多,参与较少,参与较多,使用较多,使用较多,...,False,False,True,True,True,False,False,False,False,False


In [12]:
for c in circles:
    dt[c] = dt[c].map({
        '不参与': 1, 
        '参与较少': 2,
        '参与较多': 3, 
        '参与程度极高': 4,
        '完全不使用': 1,
        '使用较少': 2,
        '使用较多': 3,
        '使用频率极高': 4,
    })

d:\anaconda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [77]:
dtm = dt[dt['gender']=='男'].sample(200)
dtf = dt[dt['gender']=='女'].sample(200)

In [78]:
# Cochran-Armitage趋势检验，A为二分类变量，B为有序分类变量
def cochran_armitage(A, B):
    x = np.array([1,2,3,4])
    y = np.array([A[B==i].sum() for i in x])
    n = np.array([A[B==i].size for i in x])
    N = n.sum()
    p = y.sum()/N
    xbar = (n*x).sum()/N
    z = (y*(x-xbar)).sum() / np.sqrt(p*(1-p)* (n*(x-xbar)**2).sum() )
    p = stats.norm.cdf(-abs(z)) * 2 * np.sign(z)
    return p

In [79]:
pm = pd.DataFrame([], columns=circles, index=reasons)
for c in circles:
    for r in reasons:
        pm[c][r] = cochran_armitage(dtm[r], dtm[c])

In [80]:
pm[abs(pm)<0.001]

,动漫,综艺,体育/电竞,明星,科技/数码,网文,游戏,微博,知乎,豆瓣,虎扑,百度贴吧,小红书,B站,抖音
抚养压力,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
生育痛苦与生育风险,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
对孩子成长环境的担忧,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
生育会影响自己实现人生价值,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
缺乏家庭照料资源,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
社会上的丁克现象（不生孩子）越来越多,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
居住环境压力,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
教育、培训费用导致经济负担,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
社会政策因素,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
降低买房成本（例如对多孩家庭给予买房上的优惠）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
pf = pd.DataFrame([], columns=circles, index=reasons)
for c in circles:
    for r in reasons:
        pf[c][r] = cochran_armitage(dtf[r], dtf[c])

In [82]:
pf[abs(pf)<0.001]

,动漫,综艺,体育/电竞,明星,科技/数码,网文,游戏,微博,知乎,豆瓣,虎扑,百度贴吧,小红书,B站,抖音
抚养压力,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
生育痛苦与生育风险,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
对孩子成长环境的担忧,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
生育会影响自己实现人生价值,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
缺乏家庭照料资源,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
社会上的丁克现象（不生孩子）越来越多,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
居住环境压力,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
教育、培训费用导致经济负担,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
社会政策因素,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
降低买房成本（例如对多孩家庭给予买房上的优惠）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00035
